# Importing Required Libararies and Packages

In [3]:
# !pip install deepface
# !pip install Deprecated
# !pip uninstall opencv-python -y
# !pip install opencv-contrib-python

In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import time

import numpy as np, pandas as pd

import cv2

# from deepface import DeepFace
from deepface_modified import DeepFace

2023-09-16 17:29:07.852101: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-16 17:29:08.604588: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Function Definitions

In [2]:
"""
    Modifiying the predefined method of the app class in InsightFace:FaceAnalysis Class, to have a label paramters to set on the image detection.
"""

def custom_draw_on(img, faces, label_lst):
    """
        Function used to draw rectangular box around the detected faces with their labels.
    """
    
    for i,face in enumerate(faces):
        box = face['facial_area']
        color = (0, 0, 255)
        cv2.rectangle(img, (box['x'], box['y']), (box['x']+box['w'], box['y']+box['h']), color, 2)
        cv2.putText(img, label_lst[i], (box['x']-1, box['y']-4),cv2.FONT_HERSHEY_COMPLEX,0.7,(0,255,0),1)

    return img

def load_label_csv(label_path)->pd.DataFrame:
    """
        Function used to load csv label file
    """
    # Load Label CSV file
    # Step: Searching for a csv file locatted in this directory
    for file in os.listdir(label_path):
        if file.endswith(".csv"): 
            csv_path = os.path.join(label_path, file)

    # Step: load the csv label file
    return pd.read_csv(csv_path)

# Face Detection

In [3]:
face_recognition_model = 'ArcFace'
face_detector_model = 'retinaface'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# Single Image Recognition

In [4]:
test_images_directory = '../data/test/'
label_path = '../data/label'
test_img = 'file.jpg'

# Load Label CSV file
df_label = load_label_csv(label_path=label_path)

detected_faces = DeepFace.extract_faces(
    img_path=test_images_directory+test_img,
    detector_backend=face_detector_model
)

found_faces = DeepFace.find(img_path=test_images_directory+test_img,
            db_path=label_path,
            model_name=face_recognition_model,
            detector_backend=face_detector_model,
            enforce_detection=True)

label_lst = []

for found_face in found_faces:
    display(found_face)
    img_label = 'Unkown'

    if len(found_face) != 0:
        # Face was detected
        # checking the ArcFace_cosine threshold
        if found_face['ArcFace_cosine'].values[0]> 0.25:
            img_label_path = found_face['identity'].values[0]
            img_label = df_label[df_label.image_name == img_label_path[img_label_path.rfind('/') + 1:]]['label'].values[0]
            
    label_lst.append(img_label)

    
img = cv2.imread(test_images_directory+test_img)

rimg= custom_draw_on(img, detected_faces, label_lst)


# Output the labeled Image
cv2.imwrite(f"../report/plots/{test_img}", rimg)

print(f"Recognitioned Faces in {test_img} are: {label_lst}. Exported image with detected faces at: ../report/plots/{test_img}")

2023-09-16 17:29:13.557037: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-16 17:29:13.585178: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


There are  1  representations found in  representations_arcface.pkl
find function lasts  3.772150754928589  seconds


,identity,source_x,source_y,source_w,source_h,ArcFace_cosine
0,../data/label/Ahmed.jpg,232,156,163,239,0.312446


Recognitioned Faces in file.jpg are: ['Ahmed']. Exported image with detected faces at: ../report/plots/file.jpg


In [15]:
test_images_directory = '../data/test/'
label_path = '../data/label'
test_img = 'file.jpg'

# Load Label CSV file
df_label = load_label_csv(label_path=label_path)

detected_faces = DeepFace.extract_faces(
    img_path=test_images_directory+test_img,
    detector_backend=face_detector_model
)

found_faces = DeepFace.find_modified(detected_faces,
            db_path=label_path,
            model_name=face_recognition_model,
            detector_backend='skip',
            enforce_detection=True)

label_lst = []

for found_face in found_faces:

    if len(found_face) != 0:
        # Face was detected
        img_label = 'UnKown'
        
        # checking the ArcFace_cosine threshold
        if found_face['ArcFace_cosine'].values[0]> 0.25:
            img_label_path = found_face['identity'].values[0]
            img_label = df_label[df_label.image_name == img_label_path[img_label_path.rfind('/') + 1:]]['label'].values[0]
            
        label_lst.append(img_label)

    else:
        label_lst.append('Unkown')
    
img = cv2.imread(test_images_directory+test_img)

rimg= custom_draw_on(img, detected_faces, label_lst)

# Output the labeled Image
cv2.imwrite(f"../report/plots/{test_img}", rimg)

print(f"Recognitioned Faces in {test_img} are: {label_lst}. Exported image with detected faces at: ../report/plots/{test_img}")

There are  1  representations found in  representations_arcface.pkl
find function lasts  0.0467832088470459  seconds
Recognitioned Faces in file.jpg are: ['Ahmed']. Exported image with detected faces at: ../report/plots/file.jpg


# Video Image

In [16]:
# Single Image Recognition Inference
def frame_recognition(
        img,
        df_label,
        model_name,
        detector_backend
        ):
    """
        Function used to detect faces and recognition them in a single Image
    """

    detected_faces = DeepFace.extract_faces(
        img_path=img,
        detector_backend=detector_backend
    )
   
    found_faces = DeepFace.find_modified(detected_faces,
                db_path=label_path,
                model_name=model_name,
                detector_backend='skip',
                enforce_detection=True)

    label_lst = []

    for found_face in found_faces:
        display(found_face)
        if len(found_face) != 0:
            # Face was detected
            img_label = 'UnKown'
            
            found_face = found_face[
                found_face.ArcFace_cosine == found_face.ArcFace_cosine.max()
                ]
            
            # checking the ArcFace_cosine threshold
            thr = 0 #0.25
            if found_face['ArcFace_cosine'].values[0]> thr:
                img_label_path = found_face['identity'].values[0]
                img_label = df_label[df_label.image_name == img_label_path[img_label_path.rfind('/') + 1:]]['label'].values[0]
                
            label_lst.append(img_label)

        else:
            label_lst.append('Unkown')
        

    rimg= custom_draw_on(img, detected_faces, label_lst)

    # To indicate the function terminated successfully
    return rimg


In [19]:
test_images_directory = '../data/test/'
label_path = '../data/label'

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# Load Label CSV file
df_label = load_label_csv(label_path=label_path)

# Create a video capture object
# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture(f"{test_images_directory}Video2.avi")

# Check if the video is opened successfully
if not cap.isOpened():
    print('Could not open the video file')
    exit()

# Set the capture buffer size
cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)
# Set the desired FPS
cap.set(cv2.CAP_PROP_FPS, 60)

# Calculate the delay time to achieve the desired frame rate
desired_fps = 60
delay_time = 1.0 / desired_fps

start_time = time.time()

# Loop through the video frames
while True:
    
    ret, frame = cap.read()

    # if time.time() - start_time > delay_time:
        
    frame = frame_recognition(frame, 
                                df_label=df_label, 
                                model_name=face_recognition_model, 
                                detector_backend=face_detector_model)
    
    cv2.imshow('frame', frame)

        # start_time = time.time()

    # Exit the loop if the user presses the 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
 
cv2.destroyAllWindows()

There are  1  representations found in  representations_arcface.pkl
find function lasts  0.24375057220458984  seconds


,identity,source_x,source_y,source_w,source_h,ArcFace_cosine


,identity,source_x,source_y,source_w,source_h,ArcFace_cosine


,identity,source_x,source_y,source_w,source_h,ArcFace_cosine


,identity,source_x,source_y,source_w,source_h,ArcFace_cosine


...

There are  1  representations found in  representations_arcface.pkl
find function lasts  0.179884672164917  seconds


,identity,source_x,source_y,source_w,source_h,ArcFace_cosine


,identity,source_x,source_y,source_w,source_h,ArcFace_cosine


,identity,source_x,source_y,source_w,source_h,ArcFace_cosine


,identity,source_x,source_y,source_w,source_h,ArcFace_cosine
